<div style="color: rgba(14, 90, 132, 1); vertical-align:middle; text-align: center;">
    <div style="font-size: 40px; font-weight: bold;">
        Validador de Cep - API VIA CEP
    </div>
    <img src="https://viacep.com.br/estatico/images/viacep.png.pagespeed.ce.I80LiA6qpr.png" align="middle" width="30%" height="30%"/>
    <div style="font-size: 20px; font-style: italic;">
    </div>
</div>

In [1]:
import pandas as pd
import requests as rs
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
# importando e formatando tabela csv
tbl = input(f'informe o caminho da tabela: ')
tabela_geral = pd.read_csv("{}".format(tbl), sep=";")
tabela_geral['BAIRRO_CORRECAO'] = ""
tabela_geral['cep'] = tabela_geral['cep'].astype("str")
tabela_geral['cep'] = tabela_geral['cep'].apply(lambda x : "0"+"{}".format(x))
tabela_geral.head(4)

informe o caminho da tabela:  C:\Users\ulissescalado\Documents\BD_REGIÃO\Tratamento_regiao.csv


,cnpj,bairro,cep,BAIRRO_CORRECAO
0,02466919/0001-09,MACEDO,07197160,
1,00871538/0001-71,VILA ISABEL,07241450,
2,02573677/0001-44,VILA MIRIAM,07190000,
3,07914734/0001-25,VILA PEDRO MOREIRA,07020001,


In [3]:
%%time
#Varredura de cep e inserção de bairros corrigidos
linha = 0
try:
    for indice in tabela_geral.iterrows():
        try:
            cep = str(tabela_geral.loc[linha,['cep']].item()).replace(".","").replace(" ","").replace("\n","").replace("cep","").replace("'","")
            time.sleep(4)
            link = rs.get(f"https://viacep.com.br/ws/{cep}/json/")
            dados = link.json()
            corr = dados.get('bairro')
            time.sleep(2)
            tabela_geral.loc[linha,['BAIRRO_CORRECAO']] =  corr
            print(f'{linha}-{corr}', end=';')
            linha += 1
        except:
            print(f'** Erro na linha: {linha}', end='**')
            linha += 1
            
finally:
    print('Finalizado com sucesso')

0-Macedo;1-Vila Izabel;2-Vila Miriam;3-Vila Pedro Moreira;4-Vila São João;5-Vila Nova Cumbica;6-Itapegica-Vila Hermínia;7-Cidade Industrial Satélite de São Paulo;8-Cidade Industrial Satélite de São Paulo;9-Vila Augusta;10-Parque Continental II;11-Vila Nova Bonsucesso;12-Vila Nova Cumbica;13-Jardim Bela Vista;14-Cidade Industrial Satélite de São Paulo;15-Vila Maricy;16-Vila Endres;17-Cumbica;18-Jardim Iporanga;19-Vila Aeroporto;20-Parque Uirapuru;21-Jardim Presidente Dutra;22-Jardim São Geraldo;23-Cidade Industrial Satélite de São Paulo;24-Vila das Bandeiras;25-Vila Venditti;26-Jardim Santa Francisca;27-Vila Nova Cumbica;28-Parque Continental III;29-Cidade Industrial Satélite de São Paulo;30-Jardim Pinhal;31-Cidade Industrial Satélite de São Paulo;32-Cidade Industrial Satélite de São Paulo;33-Água Chata;34-Vila das Bandeiras;35-Cidade Industrial Satélite de São Paulo;36-Cidade Industrial Satélite de São Paulo;37-Cidade Industrial Satélite de São Paulo;38-Jardim Santa Emilia;39-Cidade In

In [4]:
tabela_geral.head(10)

,cnpj,bairro,cep,BAIRRO_CORRECAO
0,02466919/0001-09,MACEDO,07197160,Macedo
1,00871538/0001-71,VILA ISABEL,07241450,Vila Izabel
2,02573677/0001-44,VILA MIRIAM,07190000,Vila Miriam
3,07914734/0001-25,VILA PEDRO MOREIRA,07020001,Vila Pedro Moreira
4,05978781/0001-07,DE ITAPEGICA,07044000,Vila São João
5,02661035/0001-05,VILA NOVA CUMBICA,07231000,Vila Nova Cumbica
6,33113309/0040-53,VILA ENDRES,07042040,Itapegica-Vila Hermínia
7,04441775/0001-53,CIDADE INDUSTRIAL SATELITE DE SAO PAULO,07224130,Cidade Industrial Satélite de São Paulo
8,06148424/0001-84,C.SATELITE DE CUMBIC,07223040,Cidade Industrial Satélite de São Paulo
9,02735564/0001-06,VILA AUGUSTA,07025250,Vila Augusta


In [6]:
#Varredura e correção completa, conversão Dataframe para csv
idx = tbl.index('.')
tabela_geral.to_csv("{}_correcao.csv".format(tbl[:idx]),header=True, index=True, sep=';',encoding='latin1')